In [1]:
import pandas as pd
import os
import time
import psutil
from pyspark.sql import SparkSession
from pyspark.sql.functions import pandas_udf, PandasUDFType, col
from pyspark.sql.types import StructType, StructField, StringType
import base64

# Create SparkSession
spark = SparkSession.builder.appName("DatasetEncryption").getOrCreate()
# Read the CSV file
df = spark.read.csv(r"C:\Users\91974\Documents\Bitcoin_tweets.csv", header=True, inferSchema=True)
# Choose the number of partitions
partitions = 4
df = df.repartition(partitions)

# Define the Blowfish encryption function
@pandas_udf(returnType=StructType([
    StructField("key", StringType()),
    StructField("iv", StringType())
] + [
    StructField("encrypted_" + c, StringType()) for c in df.columns
]), functionType=PandasUDFType.GROUPED_MAP)
def blowfish_encrypt(df: pd.DataFrame) -> pd.DataFrame:
    from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
    from cryptography.hazmat.primitives import padding
    import base64

    key = os.urandom(32)
    iv = os.urandom(8)
    cipher = Cipher(algorithms.Blowfish(key), modes.CBC(iv))

    def pad_and_encrypt(value):
        if value is None:
            return None
        padder = padding.PKCS7(64).padder()
        padded_data = padder.update(value.encode()) + padder.finalize()
        encryptor = cipher.encryptor()
        return base64.b64encode(encryptor.update(padded_data) + encryptor.finalize()).decode()

    # Encrypt all original columns
    start_time = time.time()
    for col in df.columns:
        if not col.startswith("encrypted_") and col not in ["key", "iv"]:
            encrypted_col = "encrypted_" + col
            df[encrypted_col] = df[col].apply(lambda x: pad_and_encrypt(x))
    end_time = time.time()

    df["key"] = base64.b64encode(key).decode()
    df["iv"] = base64.b64encode(iv).decode()

    cpu_usage = psutil.cpu_percent()
    memory_usage = psutil.virtual_memory().percent

    print(f"Encryption time: {end_time - start_time:.4f} seconds")
    print(f"CPU usage: {cpu_usage:.2f}%")
    print(f"Memory usage: {memory_usage:.2f}%")

    # Return only the encrypted columns, key, and IV
    encrypted_df = df[["key", "iv"] + ["encrypted_" + c for c in df.columns if not c.startswith("encrypted_") and c not in ["key", "iv"]]]
    return encrypted_df

# Define the Blowfish decryption function to decrypt all columns
@pandas_udf(returnType=StructType([
    StructField("decrypted_" + c, StringType()) for c in df.columns
]), functionType=PandasUDFType.GROUPED_MAP)
def blowfish_decrypt(df: pd.DataFrame) -> pd.DataFrame:
    from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
    from cryptography.hazmat.primitives import padding

    # Get the Blowfish key and IV from the first row of the DataFrame
    key = base64.b64decode(df.iloc[0]["key"])  # Decode the key
    iv = base64.b64decode(df.iloc[0]["iv"])    # Decode the IV

    # Create a Blowfish cipher object with the key
    cipher = Cipher(algorithms.Blowfish(key), modes.CBC(iv))
    def unpad_and_decrypt(value):
        if value is None:
            return None
        decryptor = cipher.decryptor()
        decrypted_data = decryptor.update(base64.b64decode(value.encode())) + decryptor.finalize()
        unpadder = padding.PKCS7(64).unpadder()
        return unpadder.update(decrypted_data) + unpadder.finalize()

    for c in df.columns:
        if c.startswith("encrypted_"):
            df["decrypted_" + c.replace("encrypted_", "")] = df[c].apply(lambda x: unpad_and_decrypt(x))
    end_time = time.time()

    cpu_usage = psutil.cpu_percent()
    memory_usage = psutil.virtual_memory().percent

    print(f"Decryption time: {end_time - start_time:.4f} seconds")
    print(f"CPU usage: {cpu_usage:.2f}%")
    print(f"Memory usage: {memory_usage:.2f}%")

    # Return only the decrypted columns
    decrypted_df = df[[c for c in df.columns if c.startswith("decrypted_")]]
    return decrypted_df

# Encrypt the dataset using Blowfish
start_time = time.time()
df_encrypted = df.groupBy().apply(blowfish_encrypt)
end_time = time.time()
encryption_time = end_time - start_time
print(f"\nAverage encryption time: {encryption_time / partitions:.4f} seconds")
print(f"Overall encryption time: {encryption_time:.4f} seconds\n")
df_encrypted.show()

# Decrypt the dataset using Blowfish
start_time = time.time()
df_decrypted = df_encrypted.groupBy().apply(blowfish_decrypt)
end_time = time.time()
decryption_time = end_time - start_time
print(f"\nAverage decryption time: {decryption_time / partitions:.4f} seconds")
print(f"Overall decryption time: {decryption_time:.4f} seconds\n")
df_decrypted.show()


C:\Users\91974\anaconda3\lib\site-packages\pyspark\sql\pandas\group_ops.py:98: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(



Average encryption time: 0.0373 seconds
Overall encryption time: 0.1492 seconds

+--------------------+------------+--------------------+-----------------------+--------------------------+
|                 key|          iv| encrypted_user_name|encrypted_user_location|encrypted_user_description|
+--------------------+------------+--------------------+-----------------------+--------------------------+
|5wo9Y6g3WKI4xjlqx...|iVktB/eh4MQ=|w72nrXGiheG9ssI9c...|                   null|                      null|
|5wo9Y6g3WKI4xjlqx...|iVktB/eh4MQ=|yNvnO5IObSbQ4mGc3...|                   null|                      null|
|5wo9Y6g3WKI4xjlqx...|iVktB/eh4MQ=|sUXjj9H00CedTBO7b...|                   null|                      null|
|5wo9Y6g3WKI4xjlqx...|iVktB/eh4MQ=|        ExFYce1yzrU=|                   null|                      null|
|5wo9Y6g3WKI4xjlqx...|iVktB/eh4MQ=|HE04VFTemy3fKHZI9...|   LyO6eOzFSP0i0qpiG...|      HE04VFTemy06F8luH...|
|5wo9Y6g3WKI4xjlqx...|iVktB/eh4MQ=|dHsVfN1UJ/rcfJCn9..